In [1]:
import os
import numpy as np
import pandas as pd
import nltk
from nltk.stem.porter import *
from nltk.tokenize import RegexpTokenizer
import random

In [2]:
def load_data():
    spam_paths = []
    ham_paths = []
    for subdir, dirs, files in os.walk('data'):
        for file in files:
            if file != 'Summary.txt' and file[-4:] == '.txt':
                file_path = os.path.join(subdir, file)
                if file_path[-8:] == 'spam.txt':
                    spam_paths.append(file_path)
                elif file_path[-7:] == 'ham.txt':
                    ham_paths.append(file_path)
                    
    dataset = [(path,1) for path in spam_paths] + [(path,0) for path in ham_paths]
    random.shuffle(dataset)
    
    
    train_ratio = 0.8
    validation_ratio = 0.1
    num_train = int(train_ratio * len(dataset))
    num_validation = int(validation_ratio * len(dataset))
    
    train_set = dataset[:num_train]
    validation_set = dataset[num_train : num_train + num_validation]
    test_set = dataset[num_train + num_validation:]
    
    train_X, train_y = zip(*train_set)
    validation_X, validation_y = zip(*validation_set)
    test_X, test_y = zip(*test_set)
    
    
    print(f'Number of files: {len(spam_paths)+len(ham_paths)}')
    print(f'Number of spam: {len(spam_paths)}')
    print(f'Number of ham: {len(ham_paths)}')
    print(f"Size of train_set: {len(train_set)}")
    print(f"Size of validation_set: {len(validation_set)}")
    print(f"Size of test_set: {len(test_set)}")
    
    return train_X, train_y, validation_X, validation_y, test_X, test_y

In [3]:
def iter_dataset(dataset_X):
    
    for filePath in dataset_X:
        file = open(filePath, 'r')
        yield file
        file.close()
        

In [4]:
def calculate_idf(corpus):
    """
    Calculates the idf for every term in the document
    """
    DF = {} #dictionary --> {word: set() of documents in which word present} key:word, value: set
    tokenizer = RegexpTokenizer(r'\w+')
    stemmer = PorterStemmer()
    num_documents = len(corpus)
    for file in iter_dataset(corpus):
        for line in file:
            words = tokenizer.tokenize(line)
            words = [stemmer.stem(word) for word in words]
            for word in words:
                try:
                    DF[word].add(file)
                except:
                    DF[word] = {file}
    
    for key in DF.keys():
        DF[key] = len(DF[key])
    
    return
                    
            

NameError: name 'train_X' is not defined

In [7]:
def train():
    idfs = calculate_idf(train_X)
    for file in train_X:
        
        #calculate the tf for the file
        
        #pass it to the logistic regression unit
                    

SyntaxError: unexpected EOF while parsing (<ipython-input-7-7d5b5e1fc198>, line 8)

In [8]:
for file in train_X:
    for line in file:
        print(line)

NameError: name 'train_X' is not defined

In [9]:
train_X, train_y, validation_X, validation_y, test_X, test_y = load_data()

train_X = iter_dataset(train_X)
validation_X = iter_dataset(validation_X)
test_X = iter_dataset(test_X)

Number of files: 33716
Number of spam: 17171
Number of ham: 16545
Size of train_set: 26972
Size of validation_set: 3371
Size of test_set: 3373


<_io.TextIOWrapper name='data/enron1/ham/0943.2000-04-14.farmer.ham.txt' mode='r' encoding='UTF-8'>

stemmedWords